In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
import pandas as pd
import keras
import keras.utils
from keras import utils as np_utils
from keras.layers import Activation,Dense,Dropout,Conv1D,Flatten,MaxPooling1D
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
import tensorflow as tf
from keras.models import Sequential
from sklearn.metrics import accuracy_score
import json
import requests
import random

Using TensorFlow backend.


In [0]:
Y_Test = pd.read_csv("/content/drive/My Drive/Classes Test.csv")
X_Test = pd.read_csv("/content/drive/My Drive/Data Test.csv")
Y_Train = pd.read_csv("/content/drive/My Drive/Classes Train.csv")
X_Train = pd.read_csv("/content/drive/My Drive/Data Train.csv")

In [0]:
X_Test = pd.DataFrame(X_Test).to_numpy()
X_Train = pd.DataFrame(X_Train).to_numpy()

In [0]:
classes_train_series = Y_Train["Name"]
classes_test_series = Y_Test["Name"]

In [0]:
#numpy unique classes
unique_classes_numpy = classes_train_series.unique()
#list unique classes
unique_classes_list = unique_classes_numpy.tolist()
#Total no of classes into List
train_classes_list = classes_train_series.to_list()

In [0]:
def classes2bow():
  vectorizer = CountVectorizer(binary=True)
  vectorizer.fit(unique_classes_list)
  return vectorizer

In [0]:
def clean_input():
  X_Training = []
  Y_Training = []
  input_data_numeric = []
  vectorizer = classes2bow()
  for x in range(0,len(X_Train)):
    X_Training.append(X_Train[x])
    label_numpy = vectorizer.transform([train_classes_list[x]]).toarray()[0]
    label_list = label_numpy.tolist()
    Y_Training.append(label_list)
  X_Training = np.array(X_Training)  
  Y_Training = np.array(Y_Training)
  return X_Training , Y_Training, vectorizer

In [0]:
X_Training , Y_Training = clean_input()


In [11]:
len(Y_Training[0])

22

In [0]:
def build_classifier():
  model = Sequential()
  model.add(Dense(units = 30,activation="relu",kernel_initializer = "uniform",input_dim = len(X_Training[0]) ))
  model.add(Dropout(rate=0.2))
  model.add(Dense(units = 30, activation="relu",kernel_initializer = "uniform"))
  model.add(Dropout(rate=0.2))
  model.add(Dense(units = len(Y_Training[0]),activation = "softmax" , kernel_initializer = "uniform"))
  model.compile(optimizer = "adam" , loss="categorical_crossentropy", metrics = ['accuracy'] )
  return model

In [13]:
model = build_classifier()
model.fit(X_Training,Y_Training ,batch_size=1,epochs=80)





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Epoch 1/80





60/60 [==============================] - 1s 15ms/step - loss: 3.1401 - acc: 0.0000e+00
Epoch 2/80
60/60 [==============================] - 0s 1ms/step - loss: 3.0814 - acc: 0.0333
Epoch 3/80
60/60 [==============================] - 0s 1ms/step - loss: 3.0599 - acc: 0.0500
Epoch 4/80
60/60 [==============================] - 0s 1ms/step - loss: 3.0531 - acc: 0.0833
Epoch 5/80
60/60 [==============================] - 0s 1ms/step - loss: 2.9597 - acc: 0.1000
Epoch 6/80
60/60 [==============================] - 0s 1ms/step - loss: 2.8380 - acc: 0.1333
Epoch 7/80
60/60 [==============================] - 0s 1ms/step - loss: 2.6893 - acc: 0.1667
Epoch 8/80
60/60 [==============================] - 0s 1ms/step - loss: 2.5371 - acc: 0.1833
Epoch 9/80
60/60 [==========

In [0]:
def cleanExample(audio_file):
  # missing audio to numeric conversion
  example = audio_file
  example = example.reshape(1,40)
  return example

In [0]:
def predictSpeaker(model,example):
  results = model.predict(example)[0]
  results = [[i,r] for i,r in enumerate(results) if r>0.25]
  results.sort(key=lambda x: x[1], reverse=True)
  return results
  

In [0]:
def numeric2class(vectorizer,model,example):
  results = predictSpeaker(model,example)
  for word , key in (vectorizer.vocabulary_).items():
    if(results[0][0] == key):
      return word
  return "NOT FOUND"    

In [0]:
audio_file = X_Training[34]
example_input = cleanExample(audio_file)

In [0]:
example_output = numeric2class(vectorizer,model,example_input)

In [35]:
print(example_output)

mahirakhan
